In [12]:
import re
import heapq 
import copy
import sys
import math

sys.setrecursionlimit(100000)

In [13]:
x1=math.radians(30)

In [14]:
1 - math.sin(math.radians(30))

0.5

In [15]:
threeDimMap = {
    (0,0) : (1,1,2),
    (30,180) : (1, 1.5, 1.866),
    (60,180) : (1, 1.866, 1.5),
    (90,180) : (1, 2, 1),
    (120,180) : (1, 1.866, 0.5),
    (150,180) : (1, 1.5, 0.144),
    (180,180) : (1, 1, 0),
    (150,0) : (1, 0.5, 0.144),
    (120,0) : (1, 0.144, 0.5),
    (90,0) : (1, 0, 1),
    (60,0) : (1, 0.144, 1.5),
    (30,0) : (1, 0.5, 1.866),
    
    #(0,0) : (1,1,2)
    (30,270) : (1.5, 1, 1.866),
    (60,270) : (1.866, 1, 1.5),
    (90,270) : (2, 1, 1),
    (120,270) : (1.866, 1, 0.5),
    (150,270) : (1.5, 1, 0.144),
    #(180,180) : (1,1,0)
    (150,90) : (0.5, 1, 0.144),
    (120,90) : (0.144, 1, 0.866),
    (90,90) : (0, 1, 1),
    (60,90) : (0.144, 1, 1.5),
    (30,90) : (0.5, 1, 1.866),
    
    #(90,0) : (1,2,1)
    (90,330) : (1.5, 1.866, 1),
    (90,300) : (1.866, 1.5, 1),
    #(90,270) : (2,1,1)
    (90,240) : (1.866, 0.866, 1),
    (90,210) : (1.5, 0.5, 1),
    #(90,180) : (1,0,1)
    (90,150) : (0.5, 0.5, 1),
    (90,120) : (0.144, 0.866, 1),
    #(90,90) : (0,1,1)
    (90,60) : (0.144, 1.5, 1),
    (90,30) : (0.5, 1.866, 1)
    
}

axis0_180Base=[(0, 0),(30, 0),(60, 0),(90, 0),(120, 0),(150, 0),(180, 180),(150, 180),(120, 180),(90, 180),(60, 180),(30, 180)]

axis90_270Base=[(0, 0),(30, 90),(60, 90),(90, 90),(120, 90),(150, 90),(180, 180),(150, 270),(120, 270),(90, 270),(60, 270),(30, 270)]

equatorBase = [(90,0),(90,30),(90,60),(90,90),(90,120),(90,150),(90,180),(90,210),(90,240),(90,270),(90,300),(90,330)]

def heuristic(X, Y):
    x = threeDimMap[X]
    y = threeDimMap[Y]
    distance = 2*math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
    return distance

    

In [16]:
heuristic((0,0), (180,180))

4.0

In [17]:
axis0_180 = {}
axis90_270 = {}
equator = {}

def populateAxes(tileNums):
    actualPos = (int(tileNums[0]), int(tileNums[1]))
    truePos = (int(tileNums[2]), int(tileNums[3]))
                
    #populate equator
    if (actualPos[0] == 90):
        equator[actualPos] = truePos
        
    #populate axis0-180
    if (actualPos[1] == 0):
        axis0_180[actualPos] = truePos
    if (actualPos[1] == 180):
        axis0_180[actualPos] = truePos
                
    #populate axis90-270
    if(actualPos[1] == 90):
        axis90_270[actualPos] = truePos
    if(actualPos[1] == 270):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 0 & actualPos[1] == 0):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 180 & actualPos[1] == 180):
        axis90_270[actualPos] = truePos
    
def readState(filename):
    file1 = open(filename,"r")
    lines = file1.readlines()
    #print (lines[1:31])
    #print("\n")
        
    for line in lines[1:31]:
        tileNums = re.findall(r'\d+', line)
        #print(tileNums)
        if(len(tileNums)>4):
            #print(tileNums[2:6])
            populateAxes(tileNums[2:6])
        else:
            #print(tileNums)
            populateAxes(tileNums)

readState("Puzzle2-4.mb")

In [18]:
class GlobeState:
    
    def __init__(self, axis0_180, axis90_270, equator, g):
        self.axis0_180 = axis0_180
        self.axis90_270 = axis90_270
        self.equator = equator
        self.g = g
        
    #rotate clockwise axis 0-180
    def RC_axis0_180(self):
        newD = {}
        
        newD[(30,180)] = self.axis0_180[(0,0)]
        
        for i in [60, 90, 120, 150, 180]:
            newD[(i, 180)] = self.axis0_180[(i-30,180)]
        
        newD[(150,0)] = self.axis0_180[(180,180)]
        
        for i in [120, 90, 60, 30, 0]:
            newD[(i, 0)] = self.axis0_180[(i+30,0)]
        
        #swaps in axis90_270    
        self.axis90_270[(0,0)] = self.axis0_180[(30,0)]
        self.axis90_270[(180,180)] = self.axis0_180[150,180]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[120,0]
        self.equator[(90,180)] = self.axis0_180[60,180]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RCC_axis0_180(self):
        newD = {}
        
        newD[(0,0)] = self.axis0_180[(30,180)]
        
        for i in [30, 60, 90, 120, 150]:
            newD[(i,0)] = self.axis0_180[(i-30,0)]
        
        newD[(180, 180)] = self.axis0_180[(150,0)]
        
        for i in [150, 120, 90, 60, 30]:
            newD[(i,180)] = self.axis0_180[(i+30, 180)]
            
        #swaps in axis90_270
        self.axis90_270[(0,0)] = self.axis0_180[(30,180)]
        self.axis90_270[(180, 180)] = self.axis0_180[(150, 0)]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[(60,0)]
        self.equator[(90,180)] = self.axis0_180[(120, 180)]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RC_axis90_270(self):
        newD = {}
        
        newD[(30,270)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,270)] = self.axis90_270[(i-30, 270)]
        
        newD[(180,180)] = self.axis90_270[(150,270)]
        newD[(150, 90)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i, 90)] = self.axis90_270[(i+30,90)]
        
        newD[(0,0)] = self.axis90_270[(30,90)]
        
        #swaps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30,90)]
        self.axis0_180[(180,180)] = self.axis90_270[(150,270)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(120,90)]
        self.equator[(90,270)] = self.axis90_270[(60,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RCC_axis90_270(self):
        newD = {}
        
        newD[(0,0)] = self.axis90_270[(30, 270)]
        newD[(30,90)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,90)] = self.axis90_270[(i-30,90)]
            
        newD[(180,180)] = self.axis90_270[(150,90)]
        newD[(150, 270)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i,270)] = self.axis90_270[(i+30, 270)]
        
        #sawps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30, 270)]
        self.axis0_180[(180,180)] = self.axis90_270[(150, 90)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(60,90)]
        self.equator[(90,270)] = self.axis90_270[(120,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RC_equator(self):
        newD = {}
        
        newD[(90,330)] = self.equator[(90,0)]
        
        for i in [300, 270, 240, 210, 180, 150, 120, 90, 60, 30, 0]:
            newD[(90, i)] = self.equator[(90, i+30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 30)]
        self.axis0_180[(90,180)] = self.equator[(90, 210)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 120)]
        self.axis90_270[(90, 270)] = self.equator[(90, 300)]
        
        #new equator
        self.equator = newD
        
    def RCC_equator(self):
        newD = {}
        
        newD[(90,0)] = self.equator[(90,330)]
        
        for i in [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]:
            newD[(90, i)] = self.equator[(90, i-30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 330)]
        self.axis0_180[(90,180)] = self.equator[(90, 150)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 60)]
        self.axis90_270[(90, 270)] = self.equator[(90, 240)]
        
        #new equator
        self.equator = newD
        
    def getCurrentHeuristic(self):
        H = 0
        
        for key in self.axis0_180.keys():
            H = H + heuristic(key, self.axis0_180[key])
        
        for key in self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.axis90_270[key])
        
        for key in self.equator.keys() - self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.equator[key])
        
        return H/12
    
    
    def isSolved(self):
        flag = True
        for key in self.axis0_180.keys():
            if (key != self.axis0_180[key]):
                flag = False
        for key in self.axis90_270.keys():
            if (key != self.axis90_270[key]):
                flag = False
        for key in self.equator.keys():
            if (key != self.equator[key]):
                flag = False
        
        return flag
    
    

In [19]:
gs = GlobeState(axis0_180, axis90_270, equator, 0)
#gs.RCC_axis0_180()

gs.isSolved()

False

In [20]:
gs1 = copy.deepcopy(gs)
gs1.equator == gs.equator
gs.getCurrentHeuristic()

4.661446366542168

In [21]:
def copyState(state):
    newState = GlobeState(state.axis0_180, state.axis90_270, state.equator, state.g)
    return newState

In [22]:
def Astar(n):
    #initialize open and closed lists
    openList=set()
    closedList=set()
    
    #k=0
    #add start state to openList
    openList.add(gs)
    
    while(len(openList)!=0):
        #print(closedList)
        #k=k+1
        #find state with min f val
        minVal=10000000000
        for state in openList:
            if(state.getCurrentHeuristic() + state.g < minVal):
                minVal = state.getCurrentHeuristic() + state.g
                q = state
        print (minVal)
        
        #q.g=q.g+(2*(math.pi)/12)
        
        #create successors
        gs1 = copy.deepcopy(q)
        gs2 = copy.deepcopy(q)
        gs3 = copy.deepcopy(q)
        gs4 = copy.deepcopy(q)
        gs5 = copy.deepcopy(q)
        gs6 = copy.deepcopy(q)
        
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
        
        #for each succesor, if successor in either list and f val in listed item is greater, skip this successor.
        #else add it to openList
        for state in [gs1, gs2, gs3, gs4, gs5, gs6]:
            if(state.isSolved()):
                return state
            
            state.g = q.g + (2*(math.pi)/12)
            f = state.g + state.getCurrentHeuristic()
            
            flag = True 
            
            if(state.g > n):
                flag=False
                
            for stateX in closedList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    #if stateX.getCurrentHeuristic() + stateX.g < f:
                    flag = False
                    print("YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY")
            
            for stateX in openList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    if state.g < stateX.g:
                        print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
                        openList.remove(stateX)
                        openList.add(state)
                    flag = False
            
            if(flag==True):
                openList.add(state)
        
        #remove q from openList
        print(len(openList))
        openList.remove(q)
        closedList.add(q)
        
        
        

In [ ]:
gh = Astar(6)

4.661446366542168
7
4.613238752982977
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11
4.7609574749128365
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
15
4.764095648996487
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
19
4.749362858201823
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
23
4.826174989172154
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
27
4.89064827052499
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
31
4.986857686224898
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
35
5.006602123703502
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
39
5.031124522846779
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
43
5.107936653817111
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
47
5.17485953217275
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
51
5.2014567783721715
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
55
5.187379240337222
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
59
5.276243350775987
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
63
5.289647855925

5.611546740575916
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
487
5.471692475332347
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
491
5.264578410259497
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
495
5.948753720607424
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
499
5.949276097437166
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
503
5.862572600852692
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
507
5.949895626543153
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
511
5.950869030513959
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
515
5.961939269523531
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
519
5.895069526948349
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
523
5.847419190493817
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
527
5.9637184339594045
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
531
5.9686525988615955
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
535
5.904137022461272
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
539
5.9754176867254545

5.872087224821737
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
952
6.2736728026972095
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
956
6.273991388077835
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
960
6.275366114273564
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
964
6.084975554137815
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
968
6.276980956627776
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
972
6.279210435222594
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
976
6.167934757320895
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
980
6.290891017928963
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
984
6.291828420430853
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
988
6.277750882395903
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
992
6.2935629817950005
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
996
6.303946126788672
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1000
6.06228712486021
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1004
6.275223361049388

6.464113503334244
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1425
6.464234065964252
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1428
6.464347443483485
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1432
6.466485895717404
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1436
6.466674010783153
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1440
6.4671692506801275
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1444
6.467278410072806
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1448
6.467723502711806
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1452
6.469705981698463
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1456
6.472027920058909
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1460
6.472551919726654
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1464
6.474621986377077
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1468
6.475555085727565
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1472
5.942437116888263
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1476
5.795

6.451179517173234
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1876
6.5642120816707585
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1880
6.1537002307428
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1884
6.150270664616986
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1888
6.5654480334478516
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1892
6.566233947524841
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1896
6.566366621178866
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1899
6.567136546946993
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1903
6.56854837365575
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1907
6.569043466842215
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1911
6.5696738090462485
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1915
6.5700093398478785
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1919
6.57245677094994
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1923
6.572865387475295
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1927
6.5735

...

7.169258886774594
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10167
7.169388861304464
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10171
7.169826621577473
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10175
7.1699840361850296
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10179
7.170000556990203
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10183
7.170074714930652
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10187
7.170158022532943
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10191
7.114516062889412
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10194
7.170255439915209
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10198
7.108622692039855
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10202
7.170546837677758
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10206
7.1708071929813135
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10210
7.17118205293545
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10214
7.132939774160096
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY

7.185609825200336
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10606
7.148705436309996
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10610
7.185635816058739
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10614
7.185728168247403
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10618
7.1858213219605584
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10622
7.185897717826872
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10626
7.185960480375904
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10630
7.185984437480375
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10634
7.186507770755228
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10638
7.18678010293603
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10642
7.186908528570708
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10646
7.186935239322494
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10650
7.187082882976181
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
10654
7.187129435077215
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY

7.201518008155548
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11029
7.201708558716367
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11033
6.928113707939234
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11037
7.201942485586317
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11041
7.202015606012305
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11045
7.202258322121052
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11049
7.202331756210304
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11052
7.202538914079719
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11056
7.202752609629124
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11060
7.2028265585706635
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11064
7.203028279359687
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11067
7.20376947500244
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11071
7.2041445456003785
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11075
7.204278776106744
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY

7.216688602447183
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11457
7.21714277995045
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11461
7.217331127425126
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11465
7.217398887173416
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11469
7.217425992155512
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11473
7.21757985540718
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11477
7.2028470646125164
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11481
7.2181550932267164
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11485
7.218168727454818
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11489
7.218446136180001
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11493
7.218536758081795
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11496
7.21927213965225
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11500
7.219289214709674
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11504
7.219309097430595
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY

7.234973063664307
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11901
7.234999533537293
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11905
7.235092852546408
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11909
7.235526478579986
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11913
7.235529609009603
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11917
7.235701399483782
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11921
7.235716876685595
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11925
6.914431070864753
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11929
7.236182415227266
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11933
7.23636806577911
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11937
7.1807261061355785
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11941
7.236597678292997
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11945
7.089654152295018
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11949
7.147107992622258
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY

In [13]:
gh.axis0_180

{(30, 180): (30, 180),
 (60, 180): (60, 180),
 (90, 180): (90, 180),
 (120, 180): (120, 180),
 (150, 180): (150, 180),
 (180, 180): (180, 180),
 (150, 0): (150, 0),
 (120, 0): (120, 0),
 (90, 0): (90, 0),
 (60, 0): (60, 0),
 (30, 0): (30, 0),
 (0, 0): (0, 0)}

In [2]:
int("inf")

ValueError: invalid literal for int() with base 10: 'inf'

In [7]:
def Astar(gs):
    k=0;
    h = []
    heapq.heappush(h, (gs.getCurrentHeuristic(), gs))
    
    while(h[0][1].isSolved() == False):    
        gs = heapq.heappop(h)[1]
        k=k+1
        print(k)
        gs1 = copy.copy(gs)
        gs2 = copy.copy(gs)
        gs3 = copy.copy(gs)
        gs4 = copy.copy(gs)
        gs5 = copy.copy(gs)
        gs6 = copy.copy(gs)
        
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
             
        a = gs1.getCurrentHeuristic()+k
        b = gs2.getCurrentHeuristic()+k
        c = gs3.getCurrentHeuristic()+k
        d = gs4.getCurrentHeuristic()+k
        e = gs5.getCurrentHeuristic()+k
        f = gs6.getCurrentHeuristic()+k
        
        if ((a,gs1) not in h):
            heapq.heappush(h, (a, gs1))
        if ((b,gs2) not in h):
            heapq.heappush(h, (b, gs2))
        if ((c,gs3) not in h):
            heapq.heappush(h, (c, gs3))
        if ((d,gs4) not in h):
            heapq.heappush(h, (d, gs4))
        if ((e,gs5) not in h):
            heapq.heappush(h, (e, gs5))
        if ((f,gs6) not in h):
            heapq.heappush(h, (f, gs6))
        #print(type(d))
    return k

In [ ]:

    
threeDimMap = {
    (0,0) : (1,1,2),
    (30,180) : (1, 0.67, 1.67),
    (60,180) : (1, 0.33, 1.33),
    (90,180) : (1, 0, 1),
    (120,180) : (1, 0.33, 0.67),
    (150,180) : (1, 0.67, 0.33),
    (180,180) : (1, 1, 0),
    (150,0) : (1, 1.33, 0.33),
    (120,0) : (1, 1.67, 0.67),
    (90,0) : (1, 2, 1),
    (60,0) : (1, 1.67, 1.33),
    (30,0) : (1, 1.33, 1.67),
    
    (30,270) : (1.33, 1, 1.67),
    (60,270) : (1.67, 1, 1.33),
    (90,270) : (2, 1, 1),
    (120,270) : (1.67, 1, 0.67),
    (150,270) : (1.33, 1, 0.33),
    (150,90) : (0.67, 1, 0.33),
    (120,90) : (0.33, 1, 0.67),
    (90,90) : (0, 1, 1),
    (60,90) : (0.33, 1, 1.33),
    (30,90) : (0.67, 1, 1.67),
    
    (90,330) : (1.33, 1.67, 1),
    (90,300) : (1.67, 1.33, 1),
    (90,240) : (1.67, 0.67, 1),
    (90,210) : (1.33, 0.33, 1),
    (90,150) : (0.67, 0.33, 1),
    (90,120) : (0.33, 0.67, 1),
    (90,60) : (0.33, 1.33, 1),
    (90,30) : (0.67, 1.67, 1)
    
}

def heuristic(X, Y):
    x = threeDimMap[X]
    y = threeDimMap[Y]
    distance = 2*math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
    return distance

    